# Agent build with LangChain agents and watsonx.data document library retrieval MCP server in stdio transport mode

#### Install the dependencies

In [ ]:
!pip install asyncio langchain langgraph langchain-mcp-adapters ibm-watson-machine-learning langchain-ibm mcp

#### Imports

In [30]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
import os
from langchain_ibm import ChatWatsonx
from pydantic import BaseModel, Field 
from typing import Optional

#### Setting up stdio server parameters 

In [ ]:
server_params = StdioServerParameters(
         command="python",
   args=[
        "-m", "ibm_watsonxdata_dl_retrieval_mcp_server",
        "--transport", "stdio"
    ],
    transport = "stdio",
        env={
            "WATSONX_DATA_API_KEY": "",
            "WATSONX_DATA_RETRIEVAL_ENDPOINT": "",
            "WATSONX_DATA_TOKEN_GENERATION_ENDPOINT": "",
            "DOCUMENT_LIBRARY_API_ENDPOINT": "",
            "LH_CONTEXT" : "",
        }
    )

#### Initialize IBM watsonx AI model for the agent

In [ ]:
parameters = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MAX_NEW_TOKENS: 1024,
    GenParams.MIN_NEW_TOKENS: 10,
    GenParams.TEMPERATURE: 0.7,
}

model = ChatWatsonx(
    model_id="meta-llama/llama-3-3-70b-instruct",
    apikey="",
    project_id="",
    url="",
    parameters=parameters
)

#### Agent response structure

In [33]:
class AgentResponse(BaseModel):
    final_answer: Optional[str] = Field(description="Detailed answer for the initial User Query if the answer was found, else provide a clear explanation of why it could not be found and suggest possible next steps or alternatives.")

#### Run MCP client

In [ ]:
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()
        tools = await load_mcp_tools(session)

        # Create and run the agent
        dl_agent = create_react_agent(model=model, tools=tools,prompt=(
                "You are a helpful question answering agent\n"
            "Your task is to answer the user query by calling the relevant tools given to you\n"),
                response_format=AgentResponse                     
                )
        
        user_input = "What is the highest invoice amount"
        
        agent_response = await dl_agent.ainvoke({
                    "messages": [
                        {
                            "role": "user",
                            "content": user_input
                        }
                    ]
                })
        structured_resp = agent_response.get('structured_response')
        if structured_resp and hasattr(structured_resp, 'final_answer'):
            print(structured_resp.final_answer)
        else:
            print(structured_resp)